# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
#adding some columns to simplfy calculations
school_data_complete["final_grade"] = (school_data_complete.reading_score + school_data_complete.math_score) / 2

school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,final_grade
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,72.5
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,77.5
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,75.0
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,62.5
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,90.5


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
total_schools = len(school_data["school_name"].unique())
total_students = len(school_data_complete["student_name"]) #one record per student, no unique student id
total_budget = school_data.budget.sum()
average_math_score = school_data_complete.math_score.mean()
average_reading_score = school_data_complete.reading_score.mean()
number_passing_reading = len(school_data_complete.loc[school_data_complete["reading_score"] >= 70])
number_passing_math = len(school_data_complete.loc[school_data_complete["math_score"] >= 70])
percent_passing_reading = number_passing_reading / total_students * 100
percent_passing_math = number_passing_math / total_students * 100
total_average_score = (average_math_score + average_reading_score) / 2

In [4]:
# Creating dataframe for summary
district_summary = pd.DataFrame({"Total_Schools": total_schools,
                                    "Total_Students": total_students,
                                    "Total_Budget": total_budget,
                                    "Average_Math_Score": average_math_score,
                                    "Average_Reading_Score": average_reading_score,
                                    "Overall_Average_Score": total_average_score,
                                    "Percent_Passing_Math": percent_passing_math,
                                    "Percent_Passing_Reading": percent_passing_reading}, index=[0])

district_summary["Total_Students"] = district_summary["Total_Students"].map("{:,}".format)
district_summary["Total_Budget"] = district_summary["Total_Budget"].map("${:,}".format)
district_summary["Average_Math_Score"] = district_summary["Average_Math_Score"].map("{:.2f}".format)
district_summary["Average_Reading_Score"] = district_summary["Average_Reading_Score"].map("{:.2f}".format)
district_summary["Overall_Average_Score"] = district_summary["Overall_Average_Score"].map("{:.2f}".format)
district_summary["Percent_Passing_Math"] = district_summary["Percent_Passing_Math"].map("{:.2f}%".format)
district_summary["Percent_Passing_Reading"] = district_summary["Percent_Passing_Reading"].map("{:.2f}%".format)


district_summary

,Total_Schools,Total_Students,Total_Budget,Average_Math_Score,Average_Reading_Score,Overall_Average_Score,Percent_Passing_Math,Percent_Passing_Reading
0,15,"39,170","$24,649,428",78.99,81.88,80.43,74.98%,85.81%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [5]:
# group data by school, type
grouped_by_school = school_data_complete.groupby(['school_name','type'])
#grouped_by_school.agg({'student_name':"count",'reading_score': np.mean,'math_score': np.mean, 'grade':np.mean})

#grouped_by_school_list = list(grouped_by_school)
#grouped_by_list
#grouped_by_school.math_score.count()
#grouped_by_school_ist

In [6]:
# https://stackoverflow.com/questions/31655634/pandas-groupy-take-only-the-first-n-groups 
#[sum(groupedSchool[1].math_score >= 70)/len(groupedSchool[1]) for groupedSchool in list(grouped_by_school)]

school_student_count = grouped_by_school["student_name"].count()
school_budget = grouped_by_school["budget"].max()
per_student_budget = school_budget / school_student_count
school_math_score = grouped_by_school.math_score.mean()
school_reading_score = grouped_by_school.reading_score.mean()
number_passing_math = [sum(each_school[1].math_score >= 70) for each_school in list(grouped_by_school)]
percent_passing_math = [(sum(each_school[1].math_score >= 70)/len(each_school[1]))*100 for each_school in list(grouped_by_school)]
percent_passing_reading = [(sum(each_school[1].reading_score >= 70)/len(each_school[1]))*100 for each_school in list(grouped_by_school)]
school_passing_rate = [((sum(each_school[1].reading_score >= 70) + sum(each_school[1].math_score >= 70)) / 2 / len(each_school[1])) * 100 for each_school in list(grouped_by_school)]

In [7]:
school_summary = pd.DataFrame({"Total Students": school_student_count,
                               "Total School Budget": school_budget,
                               "Per Student Budget": per_student_budget,
                               "Average Math Score": school_math_score,
                               "Average Reading Score": school_reading_score,
                               "% Passing Math": percent_passing_math,
                               "% Passing Reading": percent_passing_reading,
                               "Overall Passing Rate": school_passing_rate})
school_summary

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,type,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [8]:
school_summary.sort_values("Overall Passing Rate",ascending = False).head(5)

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,type,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [9]:
school_summary.sort_values("Overall Passing Rate",ascending = True).head(5)

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,type,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [10]:
#create series for each grade
school_data_indexed = school_data_complete.set_index("school_name")
ninth_grade_math = school_data_indexed.loc[school_data_indexed["grade"] == '9th', 'math_score']
tenth_grade_math = school_data_indexed.loc[school_data_indexed["grade"] == '10th', 'math_score']
eleventh_grade_math = school_data_indexed.loc[school_data_indexed["grade"] == '11th', 'math_score']
twelth_grade_math = school_data_indexed.loc[school_data_indexed["grade"] == '12th', 'math_score']

#group by school
grouped9 = ninth_grade_math.groupby('school_name')
grouped10 = tenth_grade_math.groupby('school_name')
grouped11 = eleventh_grade_math.groupby('school_name')
grouped12 = twelth_grade_math.groupby('school_name')

# Merge
#converting to data frame during merge. this cause an error on my desktop but not on my laptop
merge_9_and_10 = pd.merge(pd.DataFrame(grouped9.mean()), pd.DataFrame(grouped10.mean()), on="school_name") 
merge_9_and_10 = merge_9_and_10.rename(columns={"math_score_x":"9th Grade", "math_score_y": "10th Grade"})
merge_11_and_12 = pd.merge(pd.DataFrame(grouped11.mean()), pd.DataFrame(grouped12.mean()), on="school_name")
merge_11_and_12 = merge_11_and_12.rename(columns={"math_score_x":"11th Grade", "math_score_y": "12th Grade"})

merged_all_grades = pd.merge(merge_9_and_10, merge_11_and_12, on="school_name")
merged_all_grades

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [11]:
#create series for each grade
school_data_indexed = school_data_complete.set_index("school_name")
ninth_grade_reading = school_data_indexed.loc[school_data_indexed["grade"] == '9th', 'reading_score']
tenth_grade_reading = school_data_indexed.loc[school_data_indexed["grade"] == '10th', 'reading_score']
eleventh_grade_reading = school_data_indexed.loc[school_data_indexed["grade"] == '11th', 'reading_score']
twelth_grade_reading = school_data_indexed.loc[school_data_indexed["grade"] == '12th', 'reading_score']

#group by school
grouped9 = ninth_grade_reading.groupby('school_name')
grouped10 = tenth_grade_reading.groupby('school_name')
grouped11 = eleventh_grade_reading.groupby('school_name')
grouped12 = twelth_grade_reading.groupby('school_name')

# Merge
merge_9_and_10 = pd.merge(pd.DataFrame(grouped9.mean()), pd.DataFrame(grouped10.mean()), on="school_name")
merge_9_and_10 = merge_9_and_10.rename(columns={"reading_score_x":"9th Grade", "reading_score_y": "10th Grade"})
merge_11_and_12 = pd.merge(pd.DataFrame(grouped11.mean()), pd.DataFrame(grouped12.mean()), on="school_name")
merge_11_and_12 = merge_11_and_12.rename(columns={"reading_score_x":"11th Grade", "reading_score_y": "12th Grade"})

merged_all_grades = pd.merge(merge_9_and_10, merge_11_and_12, on="school_name")
merged_all_grades

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [12]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
spending_bin_labels = ["<$585", "$585-615", "$615-645", "$645-675"]

In [13]:
school_summary['Spending Ranges (Per Student)'] = pd.cut(school_summary['Per Student Budget'], spending_bins, labels=spending_bin_labels)

In [14]:
#group by spending bin
grouped_by_spending = school_summary.groupby('Spending Ranges (Per Student)')['Average Math Score', 'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate'].mean()

#format columns
#grouped_by_spending.rename(columns={'Spending Group':'Spending Ranges (Per Student)'})
grouped_by_spending['Average Math Score'] = grouped_by_spending['Average Math Score'].map("{:.2f}%".format)
grouped_by_spending['Average Reading Score'] = grouped_by_spending['Average Reading Score'].map("{:.2f}%".format)
grouped_by_spending['% Passing Math'] = grouped_by_spending['% Passing Math'].map("{:.2f}%".format)
grouped_by_spending['% Passing Reading'] = grouped_by_spending['% Passing Reading'].map("{:.2f}%".format)
grouped_by_spending['Overall Passing Rate'] = grouped_by_spending['Overall Passing Rate'].map("{:.2f}%".format)

grouped_by_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.46%,83.93%,93.46%,96.61%,95.04%
$585-615,83.60%,83.89%,94.23%,95.90%,95.07%
$615-645,79.08%,81.89%,75.67%,86.11%,80.89%
$645-675,77.00%,81.03%,66.16%,81.13%,73.65%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [15]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
size_bin_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
size_bins

[0, 1000, 2000, 5000]

In [16]:
school_summary['School Size'] = pd.cut(school_summary['Total Students'], size_bins, labels=size_bin_labels)

In [17]:
#group by spending bin
grouped_by_size = school_summary.groupby('School Size')['Average Math Score', 'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate'].mean()

grouped_by_size['Average Math Score'] = grouped_by_size['Average Math Score'].map("{:.2f}%".format)
grouped_by_size['Average Reading Score'] = grouped_by_size['Average Reading Score'].map("{:.2f}%".format)
grouped_by_size['% Passing Math'] = grouped_by_size['% Passing Math'].map("{:.2f}%".format)
grouped_by_size['% Passing Reading'] = grouped_by_size['% Passing Reading'].map("{:.2f}%".format)
grouped_by_size['Overall Passing Rate'] = grouped_by_size['Overall Passing Rate'].map("{:.2f}%".format)

grouped_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.82%,83.93%,93.55%,96.10%,94.82%
Medium (1000-2000),83.37%,83.86%,93.60%,96.79%,95.20%
Large (2000-5000),77.75%,81.34%,69.96%,82.77%,76.36%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [18]:
#group by spending bin
grouped_by_type = school_summary.groupby('type')['Average Math Score', 'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate'].mean()

grouped_by_type['Average Math Score'] = grouped_by_type['Average Math Score'].map("{:.2f}%".format)
grouped_by_type['Average Reading Score'] = grouped_by_type['Average Reading Score'].map("{:.2f}%".format)
grouped_by_type['% Passing Math'] = grouped_by_type['% Passing Math'].map("{:.2f}%".format)
grouped_by_type['% Passing Reading'] = grouped_by_type['% Passing Reading'].map("{:.2f}%".format)
grouped_by_type['Overall Passing Rate'] = grouped_by_type['Overall Passing Rate'].map("{:.2f}%".format)

grouped_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,
Charter,83.47%,83.90%,93.62%,96.59%,95.10%
District,76.96%,80.97%,66.55%,80.80%,73.67%
